<a href="https://colab.research.google.com/github/nerobite/neural_networks/blob/main/Content_based_recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import pandas as pd
import numpy as np
from datetime import datetime

from tqdm import tqdm_notebook
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!wget https://github.com/ALKONDR/netology-recsys/archive/refs/heads/master.zip
!unzip master.zip

--2024-01-31 06:30:33--  https://github.com/ALKONDR/netology-recsys/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ALKONDR/netology-recsys/zip/refs/heads/master [following]
--2024-01-31 06:30:33--  https://codeload.github.com/ALKONDR/netology-recsys/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip.5’

master.zip.5            [  <=>               ]   1.06M  5.23MB/s    in 0.2s    

2024-01-31 06:30:33 (5.23 MB/s) - ‘master.zip.5’ saved [1111929]

Archive:  master.zip
dfe2a910caf170a1f0fd2174867169ce737c9dc7
replace netology-recsys-master/lecture-1/.ipynb_checkpoint

In [ ]:
prefix = 'netology-recsys-master/lecture-1'

links = pd.read_csv(os.path.join(prefix, 'links.csv'))
movies = pd.read_csv(os.path.join(prefix, 'movies.csv'))
ratings = pd.read_csv(os.path.join(prefix, 'ratings.csv'))
tags = pd.read_csv(os.path.join(prefix, 'tags.csv'))

In [ ]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
movies_ratings = movies.join(ratings.set_index('movieId'), on='movieId')

In [ ]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [ ]:
movie_genres = movies.genres.apply(change_string).tolist()

In [ ]:
movie_genres[:10]

['Adventure Animation Children Comedy Fantasy',
 'Adventure Children Fantasy',
 'Comedy Romance',
 'Comedy Drama Romance',
 'Comedy',
 'Action Crime Thriller',
 'Comedy Romance',
 'Adventure Children',
 'Action',
 'Action Adventure Thriller']

In [ ]:
movies['num_generes'] = movies['genres'].apply(lambda x: len(x.split('|')))
movies.head()

,movieId,title,genres,num_generes
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
1,2,Jumanji (1995),Adventure|Children|Fantasy,3
2,3,Grumpier Old Men (1995),Comedy|Romance,2
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3
4,5,Father of the Bride Part II (1995),Comedy,1


In [ ]:
movies_ratings

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09
...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09
9739,193585,Flint (2017),Drama,184.0,3.5,1.537110e+09
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184.0,3.5,1.537110e+09


In [ ]:
#преобразуем данные в векторы
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(movie_genres)
X_train_tfidf

<9742x20 sparse matrix of type '<class 'numpy.float64'>'
	with 22084 stored elements in Compressed Sparse Row format>

In [ ]:
genre_df = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf.get_feature_names_out())
genre_df

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,0.0,0.000000,0.482990,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,0.0,0.000000,0.593662,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,0.0,0.466405,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,0.0,0.000000,0.575034,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9738,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,0.0,0.000000,0.638968,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9739,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9740,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
movies_ratings1 = movies.join(genre_df)

In [ ]:
movies_ratings1

,movieId,title,genres,num_generes,action,adventure,animation,children,comedy,crime,...,horror,imax,musical,mystery,nogenreslisted,romance,scifi,thriller,war,western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,3,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,2,0.000000,0.000000,0.000000,0.000000,0.570915,0.0,...,0.0,0.0,0.0,0.0,0.0,0.821009,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3,0.000000,0.000000,0.000000,0.000000,0.505015,0.0,...,0.0,0.0,0.0,0.0,0.0,0.726241,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,1,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,4,0.436010,0.000000,0.614603,0.000000,0.318581,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,3,0.000000,0.000000,0.682937,0.000000,0.354002,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9739,193585,Flint (2017),Drama,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2,0.578606,0.000000,0.815607,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
#Найдем ближайших соседей
neigh = NearestNeighbors(n_neighbors=7, metric='euclidean')
neigh.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_neighbors=7)

In [ ]:
test = change_string("Adventure|Comedy|Fantasy|Documentary")

X_tfidf = tfidf.transform([test])

res = neigh.kneighbors(X_tfidf, return_distance=True)

In [ ]:
res

(array([[0.38758619, 0.56525059, 0.58114553, 0.65617877, 0.65617877,
         0.66834044, 0.66834044]]),
 array([[8014, 5836, 8161, 7597, 4853,  863, 3376]]))

In [ ]:
movies_with_tags = movies_ratings1.merge(tags, on='movieId')

In [ ]:
movies_with_tags

,movieId,title,genres,num_generes,action,adventure,animation,children,comedy,crime,...,mystery,nogenreslisted,romance,scifi,thriller,war,western,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,336,pixar,1139045764
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,474,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,0.000000,0.416846,0.516225,0.504845,0.267586,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,567,fun,1525286013
3,2,Jumanji (1995),Adventure|Children|Fantasy,3,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,62,fantasy,1528843929
4,2,Jumanji (1995),Adventure|Children|Fantasy,3,0.000000,0.512361,0.000000,0.620525,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,62,magic board game,1528843932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,4,0.418252,0.476072,0.000000,0.576575,0.000000,0.0,...,0.0,0.0,0.0,0.515735,0.0,0.0,0.0,62,star wars,1528934552
3679,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,4,0.445376,0.000000,0.627805,0.000000,0.325424,0.0,...,0.0,0.0,0.0,0.549181,0.0,0.0,0.0,184,anime,1537098582
3680,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,4,0.445376,0.000000,0.627805,0.000000,0.325424,0.0,...,0.0,0.0,0.0,0.549181,0.0,0.0,0.0,184,comedy,1537098587
3681,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,4,0.445376,0.000000,0.627805,0.000000,0.325424,0.0,...,0.0,0.0,0.0,0.549181,0.0,0.0,0.0,184,gintama,1537098603


In [ ]:
#преобразуем теги в векторы
movies_with_tags[movies_with_tags.title == 'Toy Story (1995)']

,movieId,title,genres,num_generes,action,adventure,animation,children,comedy,crime,...,mystery,nogenreslisted,romance,scifi,thriller,war,western,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,0.0,0.416846,0.516225,0.504845,0.267586,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,336,pixar,1139045764
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,0.0,0.416846,0.516225,0.504845,0.267586,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,474,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,0.0,0.416846,0.516225,0.504845,0.267586,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,567,fun,1525286013


In [ ]:
movies_with_tags.tag.unique()

array(['pixar', 'fun', 'fantasy', ..., 'star wars', 'gintama', 'remaster'],
      dtype=object)

In [ ]:
movies_with_tags.tag.unique().shape

(1589,)

In [ ]:
movies_with_tags.dropna(inplace=True)

In [ ]:
movies_with_tags.title.unique().shape

(1572,)

In [ ]:
def change_string(s):
    return str(s).replace(' ', '').replace('-', '').lower()

tag_strings = []
movies_list = []

for movie, group in tqdm(movies_with_tags.groupby('movieId')):
    tag_strings.append(' '.join([change_string(s) for s in group.tag.values]))
    movies_list.append(movie)

  0%|          | 0/1572 [00:00<?, ?it/s]

In [ ]:
tag_strings[:10]

['pixar pixar fun',
 'fantasy magicboardgame robinwilliams game',
 'moldy old',
 'pregnancy remake',
 'remake',
 'politics president',
 'politics president',
 'mafia',
 'janeausten',
 'hollywood']

In [ ]:
movies_with_tags[movies_with_tags['title'] == '(500) Days of Summer (2009)']

,movieId,title,genres,num_generes,action,adventure,animation,children,comedy,crime,...,mystery,nogenreslisted,romance,scifi,thriller,war,western,userId,tag,timestamp
3078,69757,(500) Days of Summer (2009),Comedy|Drama|Romance,3,0.0,0.0,0.0,0.0,0.505015,0.0,...,0.0,0.0,0.726241,0.0,0.0,0.0,0.0,477,artistic,1279956134
3079,69757,(500) Days of Summer (2009),Comedy|Drama|Romance,3,0.0,0.0,0.0,0.0,0.505015,0.0,...,0.0,0.0,0.726241,0.0,0.0,0.0,0.0,477,Funny,1279956141
3080,69757,(500) Days of Summer (2009),Comedy|Drama|Romance,3,0.0,0.0,0.0,0.0,0.505015,0.0,...,0.0,0.0,0.726241,0.0,0.0,0.0,0.0,477,humorous,1279956124
3081,69757,(500) Days of Summer (2009),Comedy|Drama|Romance,3,0.0,0.0,0.0,0.0,0.505015,0.0,...,0.0,0.0,0.726241,0.0,0.0,0.0,0.0,477,inspiring,1279956130
3082,69757,(500) Days of Summer (2009),Comedy|Drama|Romance,3,0.0,0.0,0.0,0.0,0.505015,0.0,...,0.0,0.0,0.726241,0.0,0.0,0.0,0.0,477,intelligent,1279956132
3083,69757,(500) Days of Summer (2009),Comedy|Drama|Romance,3,0.0,0.0,0.0,0.0,0.505015,0.0,...,0.0,0.0,0.726241,0.0,0.0,0.0,0.0,477,quirky,1279956145
3084,69757,(500) Days of Summer (2009),Comedy|Drama|Romance,3,0.0,0.0,0.0,0.0,0.505015,0.0,...,0.0,0.0,0.726241,0.0,0.0,0.0,0.0,477,romance,1279956123
3085,69757,(500) Days of Summer (2009),Comedy|Drama|Romance,3,0.0,0.0,0.0,0.0,0.505015,0.0,...,0.0,0.0,0.726241,0.0,0.0,0.0,0.0,477,Zooey Deschanel,1279956120


In [ ]:
tfidf_tag = TfidfVectorizer()
X_train_tfidf_tag = tfidf_tag.fit_transform(tag_strings)
X_train_tfidf_tag

<1572x1472 sparse matrix of type '<class 'numpy.float64'>'
	with 3598 stored elements in Compressed Sparse Row format>

In [ ]:
tag_df = pd.DataFrame(X_train_tfidf_tag.toarray(), columns=tfidf_tag.get_feature_names_out())
tag_df

,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001like,2danimation,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
movies_with_tags1 = movies_with_tags.merge(tag_df)
movies_with_tags1

,movieId,title,genres,num_generes,action,adventure,animation,children,comedy,crime,...,worldwari,worldwarii,writing,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel
0,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,104,Happy Gilmore (1996),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9130,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9131,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9132,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
raiting_user_m = ratings[['rating']].groupby(ratings['userId']).mean()
raiting_user_m.rename(columns={'rating': 'rating_mean'}, inplace=True)
raiting_user_m.reset_index(inplace=True)
raiting_user_m = ratings.merge(raiting_user_m)
raiting_user_m
#средний рейтинг для пользователя

,userId,movieId,rating,timestamp,rating_mean
0,1,1,4.0,964982703,4.366379
1,1,3,4.0,964981247,4.366379
2,1,6,4.0,964982224,4.366379
3,1,47,5.0,964983815,4.366379
4,1,50,5.0,964982931,4.366379
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,3.688556
100832,610,168248,5.0,1493850091,3.688556
100833,610,168250,5.0,1494273047,3.688556
100834,610,168252,5.0,1493846352,3.688556


In [ ]:
raiting_user_m

,userId,movieId,rating,timestamp,rating_mean
0,1,1,4.0,964982703,4.366379
1,1,3,4.0,964981247,4.366379
2,1,6,4.0,964982224,4.366379
3,1,47,5.0,964983815,4.366379
4,1,50,5.0,964982931,4.366379
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,3.688556
100832,610,168248,5.0,1493850091,3.688556
100833,610,168250,5.0,1494273047,3.688556
100834,610,168252,5.0,1493846352,3.688556


In [ ]:
raiting_user_median = ratings[['rating']].groupby(ratings['userId']).median()
raiting_user_median.rename(columns={'rating': 'rating_median'}, inplace=True)
raiting_user_median.reset_index(inplace=True)
raiting_user_median = ratings.merge(raiting_user_median)
raiting_user_median

,userId,movieId,rating,timestamp,rating_median
0,1,1,4.0,964982703,5.0
1,1,3,4.0,964981247,5.0
2,1,6,4.0,964982224,5.0
3,1,47,5.0,964983815,5.0
4,1,50,5.0,964982931,5.0
...,...,...,...,...,...
100831,610,166534,4.0,1493848402,3.5
100832,610,168248,5.0,1493850091,3.5
100833,610,168250,5.0,1494273047,3.5
100834,610,168252,5.0,1493846352,3.5


In [ ]:
rating_m = raiting_user_m.merge(raiting_user_median)

In [ ]:
rating_m = rating_m.drop(['userId', "timestamp"], axis=1)
rating_m

,movieId,rating,rating_mean,rating_median
0,1,4.0,4.366379,5.0
1,3,4.0,4.366379,5.0
2,6,4.0,4.366379,5.0
3,47,5.0,4.366379,5.0
4,50,5.0,4.366379,5.0
...,...,...,...,...
100831,166534,4.0,3.688556,3.5
100832,168248,5.0,3.688556,3.5
100833,168250,5.0,3.688556,3.5
100834,168252,5.0,3.688556,3.5


In [ ]:
movies_with_tags2 = movies_with_tags1.merge(rating_m, on='movieId')

In [ ]:
movies_with_tags2

,movieId,title,genres,num_generes,action,adventure,animation,children,comedy,crime,...,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel,rating,rating_mean,rating_median
0,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.493631,3.0
1,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.920000,4.0
2,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.552632,5.0
3,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.875940,4.0
4,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.901786,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40598,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.846154,5.0
40599,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.245455,2.0
40600,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,4.365385,4.5
40601,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,2.642050,2.5


In [ ]:
rating = ratings[['rating']].groupby(ratings['movieId']).mean()

In [ ]:
rating

,rating
movieId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429
...,...
193581,4.000000
193583,3.500000
193585,3.500000


In [ ]:
rating.reset_index(inplace=True)

In [ ]:
rating

,movieId,rating
0,1,3.920930
1,2,3.431818
2,3,3.259615
3,4,2.357143
4,5,3.071429
...,...,...
9719,193581,4.000000
9720,193583,3.500000
9721,193585,3.500000
9722,193587,3.500000


In [ ]:
movies

,movieId,title,genres,num_generes
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5
1,2,Jumanji (1995),Adventure|Children|Fantasy,3
2,3,Grumpier Old Men (1995),Comedy|Romance,2
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3
4,5,Father of the Bride Part II (1995),Comedy,1
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,4
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,3
9739,193585,Flint (2017),Drama,1
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2


In [ ]:
movies_df = movies_with_tags2.merge(rating, on='movieId')

In [ ]:
movies_df

,movieId,title,genres,num_generes,action,adventure,animation,children,comedy,crime,...,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel,rating_x,rating_mean,rating_median,rating_y
0,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.493631,3.0,3.071429
1,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.920000,4.0,3.071429
2,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.552632,5.0,3.071429
3,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.875940,4.0,3.071429
4,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.901786,4.0,3.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40598,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.846154,5.0,3.805556
40599,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.245455,2.0,3.805556
40600,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.5,4.365385,4.5,3.805556
40601,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.5,2.642050,2.5,3.805556


In [ ]:
y = movies_df['rating_y']

In [ ]:
y.dropna(inplace=True)

In [ ]:
y

0        3.071429
1        3.071429
2        3.071429
3        3.071429
4        3.071429
           ...   
40598    3.805556
40599    3.805556
40600    3.805556
40601    3.805556
40602    3.805556
Name: rating_y, Length: 40603, dtype: float64

In [ ]:
df = movies_df.iloc[y.index]

In [ ]:
df

,movieId,title,genres,num_generes,action,adventure,animation,children,comedy,crime,...,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel,rating_x,rating_mean,rating_median,rating_y
0,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.493631,3.0,3.071429
1,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.920000,4.0,3.071429
2,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.552632,5.0,3.071429
3,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.875940,4.0,3.071429
4,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.901786,4.0,3.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40598,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.846154,5.0,3.805556
40599,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.245455,2.0,3.805556
40600,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.5,4.365385,4.5,3.805556
40601,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.5,2.642050,2.5,3.805556


In [ ]:
df.fillna(0, inplace=True)

In [ ]:
df

,movieId,title,genres,num_generes,action,adventure,animation,children,comedy,crime,...,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel,rating_x,rating_mean,rating_median,rating_y
0,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.493631,3.0,3.071429
1,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.920000,4.0,3.071429
2,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.552632,5.0,3.071429
3,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.875940,4.0,3.071429
4,5,Father of the Bride Part II (1995),Comedy,1,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.901786,4.0,3.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40598,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.846154,5.0,3.805556
40599,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.245455,2.0,3.805556
40600,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.5,4.365385,4.5,3.805556
40601,176371,Blade Runner 2049 (2017),Sci-Fi,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.5,2.642050,2.5,3.805556


In [ ]:
numeric_columns = df.select_dtypes(include='number').columns

In [ ]:
numeric_columns

Index(['movieId', 'num_generes', 'action', 'adventure', 'animation',
       'children', 'comedy', 'crime', 'documentary', 'drama',
       ...
       'wry', 'youngermen', 'zither', 'zoekazan', 'zombies', 'zooeydeschanel',
       'rating_x', 'rating_mean', 'rating_median', 'rating_y'],
      dtype='object', length=1483)

In [ ]:
df = df[numeric_columns]

In [ ]:
df

,movieId,num_generes,action,adventure,animation,children,comedy,crime,documentary,drama,...,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel,rating_x,rating_mean,rating_median,rating_y
0,5,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.493631,3.0,3.071429
1,5,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.920000,4.0,3.071429
2,5,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.552632,5.0,3.071429
3,5,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.875940,4.0,3.071429
4,5,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.901786,4.0,3.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40598,176371,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.846154,5.0,3.805556
40599,176371,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.245455,2.0,3.805556
40600,176371,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.5,4.365385,4.5,3.805556
40601,176371,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.5,2.642050,2.5,3.805556


In [ ]:
df = df.drop(['movieId', 'num_generes', 'rating_y'], axis=1)
df

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,filmnoir,...,wrongfulimprisonment,wry,youngermen,zither,zoekazan,zombies,zooeydeschanel,rating_x,rating_mean,rating_median
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3.493631,3.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.920000,4.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.552632,5.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.875940,4.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.901786,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.846154,5.0
40599,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2.245455,2.0
40600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,4.365385,4.5
40601,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,2.642050,2.5


In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
df1= scaler.fit_transform(df) #маштабируем данные

In [ ]:
#Поделим данные на обучающую выборку и тестовую. Указываем в test_size долю датасета, которая пойдет на тестовую выбору
X_train, X_test, y_train, y_test = train_test_split(df1, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()

In [ ]:
# обучаем на части датасета (train)
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
model.score(X_test,y_test)

0.3492855210683471

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model2 = DecisionTreeRegressor()

In [ ]:
model2.fit(X_train, y_train)

DecisionTreeRegressor()

In [ ]:
model2.score(X_test,y_test)

0.9954638676921634

In [ ]:
y_pred = model2.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred)) #LinearRegression
print('RMSE:', rmse)

RMSE: 0.3514212422821256


In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred)) #DecisionTreeRegressor
print('RMSE:', rmse)

RMSE: 0.02934104863198965
